<a href="https://colab.research.google.com/github/drwbkr1/Grad504-Hierarchical-Cluster-Project/blob/main/GRAD504_HC_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Descriptive Modeling: Hierarchical Clustering with HDBSCAN
In many real-world applications, datasets arrive without labels. This information isn't used for making predictions. Rather, they're particularly helpful for understanding relationships between entities.

Descriptive modeling allows us to group datapoints into clusters, which tell us more about their structure and density. We often see this type of machine learning used in land-use zoning or image segmentation. In this dataset, we'll discover relationships between flowers as they are found across North America.

In the implementation below, I demonstrate handling this sparse, high-dimensional dataset. I utilize a  Hierarchical Density-Based Spatial Clustering of Applications with Noise (HDBSCAN) algorithm, which builds a weighted minimum spanning tree (MST) for improved efficiency, to agglomeratively cluster flowers. With it, I test multiple distance metrics to gain varying insights on flower and cluster relationships.

NOTE: due to a miscommunication in assignment parameters, my implementation features the HDBSCAN tool instead of granular scripting. As such, please receive my detailed explanation of the tool as an alternative to what might have described those granular functions.

In [9]:
import pandas as pd
import numpy as np
import time
from sklearn.preprocessing import MultiLabelBinarizer

plants= ("/content/drive/MyDrive/colab_notebooks/datasets/plants.data")


After discovering the correct encoding, I noticed the loader didn't like the
different number of datapoints in each line. So now I'm one-hot encoding the
dataset right out of the box

We're going to split the plants from the states, then one-hot encode the states.
We'll add the plants back to the dataframe after the one-hot encoding.

In [4]:
plants_list= []
states_list= []

with open(plants, encoding='ISO-8859-1') as file:
  for line in file:
    items= line.strip().split(",") #turn each line into a list
    plants_list.append(items[0])            #reorganize the list into respective variables
    states_list.append(items[1:])

#states_list[:10]


In [5]:
mlb= MultiLabelBinarizer()

states_ohe= mlb.fit_transform(states_list)

df= pd.DataFrame(states_ohe, columns= mlb.classes_, index= plants_list)
df= df.rename(columns= {"index" : "Flowers"})
#df.head(10)

In [6]:
df.shape

(34781, 70)

In [ ]:
'''Oh boy, 34000 flower types in 70 regions. That's 2.38 million combinations.
We have our work cut out for us'''

#HDBSCAN: A Density-Based, Hierarchical Clustering Tool
In the following codeblocks, I import HDBSCAN from the hdbscan library. This tool loads a hierarchical clustering algorithm that balances single-linkage clustering with the Mutual Reachability Distance (MRD) to build an agglomerative MST, representing tight groups that converge per event lamdas. After condensing the tree with hyperparameters (min_cluster_size), HDBSCAN selects clusters using an Excess of Mass rule. Let's take a closer look at each of these elements.

In [7]:
from hdbscan.hdbscan_ import HDBSCAN

## 1. Calculate Core Distances
HDB uses a K-Nearest Neighbors algorithm to calculate core distances between datapoints. Lower numbers represent higher density. It receives two hyperparameters to calculate this number:
- min_samples= k
- metric= 'distance metric'

For example, if min_samples= 5, the core distance would equal the metric distance from the 5th nearest neighbor.
## 2. Calculate Mutual Reachability Distance
MRD is the buffer that allows the MST to reduce computations; it's going to widen the distance between points based on the size of the core distance. Essentially, for any distance between any two points, if either of them has a higher core value, that will replace their distance. This acts as a weight that "pushes" outliers further away, reducing the chaining that might occur in the MST.

HDBSCAN equates the MRD using the formula:

MRD(i,j)=max{core(i), core(j), d(i,j)}.
## 3. Build Minimim Spanning Tree
This is the bread and butter of HDBSCAN. A minimum spanning tree begins with each point as its own tree---Agglomerative. Then, it iteratively merges points based on sorted MRDs, beginning with the lowest---Single Linkage. As points merge and the MST reaches larger and larger MRDs we see increasingly bigger groups of points merging. This continues until all points are connected in one tree.

This is important because it eliminates the (n^2) calculations that would otherwise occur when determining clusters. Because the MST avoids cyclical edges, calculations are much closer to (n log n).

This step of HDBSCAN utilizes either [Prim's](https://en.wikipedia.org/wiki/Prim%27s_algorithm) or [Baruvka's](https://en.wikipedia.org/wiki/Bor%C5%AFvka%27s_algorithm) algorithm to assign edges. These are responsible for increasing speed within HDBSCAN. They are applied automatically using the hyperparameter:
- algorithm= 'best' #default

where 'best' chooses the ideal method depending on distance metrics and dimensionality.

## 4. Sweep Thresholds
After building the MST, HDBSCAN sweeps the entire tree for event lamdas. These represent the moment where two points (or groups of points) merge. each threshold is detemined as such:

λ(e)​=1/MRD(e)

where λ(e​) is the event where the merge occurs and MRD is the mutual reachability distance calculated in step 2. in many cases, points featue similar MRDs; the more frequent an MRD occurs, the larger that merge will look.

## 5. Condense Hierarchy
We now have our completed hierarchically-clustered dataset. From here, we can select our clusters or prune to find specific relationships.

The assignment has a objective to discover the 5 tightest clusters with a minimum size of 1000 flowers. HDBSCAN allows us to condense our MST using the parameter 'min_cluster_size' so when we set it to 1000, our tree only includes groups that would include this size. The resto of those points become noise.

## 6. Select Clusters
By default HDBSCAN uses the Excess of Mass rule to segment groups. First, we calculate the stabilities of parent and children clusters. we do this by subtracting the event lambda where the children clusters birthed from the event lambda where the clusters split. We take this difference and multiply it by the number of points within the cluster.

We take this stability for both the parent and the children clusters. If the sum of the similarity scores are larger than the parent's similarity scores, we select the child clusters in our analysis.






# Initial Implementation: Jaccard Distance
As you'll see below, choosing a metric turned out to be one of the more complex and important decisions in this impementation. However, to get started, I used the first metric that made sense: Jaccard.

This measure takes the similarities between two points and divides them by all possible features. For example, if flower A and flower B are found in 10 similar states, and there are 70 states in the dataset it would have a similarity score of 1/7. It's a highly intuitive metric when using binary data.

Let's take a look at how Jaccard influences:

core distance -> MRD -> MST -> clusters

In [11]:
t0= time.time()
clusterer= HDBSCAN(min_cluster_size= 1000,
                   min_samples=3,
                   metric= 'jaccard',
                   core_dist_n_jobs= -1
                   )

clusterer.fit(df)
t1= time.time()
execution= t1-t0
print(f'Execution time: {execution} seconds')

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Execution time: 265.4992823600769 seconds


One of the major benefits of using HDBSCAN is its reduced inference time. Because it limits the number of edge calculations with an MST, the algorithm was able to select a number of clusters with at least 1000 flowers each in just over four minutes.

Let's take a look at our selection.

In [12]:
labels= clusterer.labels_

unique_labels, count= np.unique(labels, return_counts= True)
for label, cnt in zip(unique_labels, count):
  name= (f"Cluster {label}") if label != -1 else "Noise"
  print(f"{name}: {cnt} points")

Noise: 4035 points
Cluster 0: 1582 points
Cluster 1: 1896 points
Cluster 2: 1113 points
Cluster 3: 22735 points
Cluster 4: 3420 points


On our first try, we received exactly five clusters, meeting the objective of our project and ending the experiment right there. It's just that easy.

Okay, maybe it's not exactly that easy; for instance, what do those clusters even represent? Why does cluster 3 have 22,000 points? And why does it seem like there's so much noise?

## Cluster Representation
We can learn a lot about a cluster, its flowers and the representation of Jaccard by looking at its composition. I'm particularly interested in a few things.

For one, which states are the most prominent? Are there multiple? Being able to see this gives me an idea of variance. It may feel intuitive if secondary states are also geographically near the primary state. If it's more spread out, what questions should we ask from there?

Next, I'm interested in the nodes within the cluster. For example, how many states does each flower show up in? This ties into the first study---If a flower is found in many states, is it okay that the cluster is wide? Is it still tightly represented?

Let's see what this megacluster 3 looks like on the inside.

In [23]:
cluster_id = 4  #We're going to look at a specific cluster with this.

mask= (labels == cluster_id)
df_clust= df.loc[mask]

state_counts= df_clust.sum(axis= 0)  #Series indexed by state code
state_counts_sorted= state_counts.sort_values(ascending= False)

print(f"State counts for Cluster {cluster_id}:")
print(state_counts_sorted[:])

flower_counts= df.loc[mask].sum(axis= 1)  #Series indexed by flower name
print(flower_counts.nlargest(len(flower_counts)).describe())


State counts for Cluster 4:
ca    3420
dc       1
pe       1
al       0
ab       0
      ... 
wa       0
wi       0
wv       0
wy       0
yt       0
Length: 70, dtype: int64
count    3420.000000
mean        1.000585
std         0.024179
min         1.000000
25%         1.000000
50%         1.000000
75%         1.000000
max         2.000000
dtype: float64


Initially, I thought I might see flowers primarily located in one state and expanding from there. What this shows me, however, are flowers located in many states, with commonalities across the board.

What's more, when I look at the description, I see a max of 69 (a flower located in all but two states) and a mean of 13 (on average the 22,000 flowers are located in 13 states).

What does this tell me about the cluster? It seems that it represents flowers that are more common. But if it contains lots of flowers from all over, what does that set space look like with the other clusters?

Without expanding the code, here's what they look like:
- Cluster 0: 1582 points, pr= 1582, ab= 0, al= 0
- Cluster 1: 1896 points, hi= 1896, ab= 0, al= 0
- Cluster 2: 1113 points, tx= 1113, il= 1, ab= 0
- Cluster 4: 3420 points, ca= 3420, dc= 1, pe= 1

So overall, this cluster set represents:

1. all the common flowers across the continent without chaining too much, and
2. the few states that have enough flowers to represent a cluster on their own
## Why is this happening?
*tl:dr It's highly more likely that a 70-D flower is similar to a 69-D flower than a 2-D flower is similar to a 1-D flower within the 70-D set space.*

It's a super curious split, but I presume its due to the way Jaccard handles similarity. To visualize this, let's try to compare what our mind's eye wants to see---clusters overlayed on a map of North America---to what is actually occuring---clusters spread over a 70-dimensional set space.

Instead of the data looking like pieces of a puzzle (that 2-D map), each state-sum for each flower represents something like a set of its own: "here are all the flowers located in 70 states, and here are all the flowers located in 69 states, and here are all the flowers located in 68 states..." These sets are grouped closer together as they sit on nearby planes.

However, as the flowers become more and more sporatically spread across different states, they leave that zone of influence. We start to see them floating around on the edges of the 70-D space not really relating to anything until the space begins to converge again. When the sets start to reach the range of "and here are all the flowers located in this 3-state range, and here are all the flowers located in this 2-state range, and here are all the flowers in this one-state range" We start to see the flowers able to relate again.

I think it helps to explain what's happening noise portion. It includes any points that would disproportionalize the mega cluster or the micro clusters.

I believe that we would see several more regional clusters---such as florida-georgia-alabama---if the cluster sizes could be smaller. For example, 'sabal palmetto' is a regional flower located in North Carolina and 6 surrounding states. It could represent a smaller cluster, but added to a larger one, it only widens the cluster space. That's what every other pseudo cluster would be doing. Unless there were 1000 flowers found in a 3-state region (excluding all other states), they only act to disrupt a cluster structure.

And this is likely why we see clusters at the extremes. It's a lot easier to relate a 70-state flower to a 69-state flower. It's much harder to relate a 3-state flower to a 2-state flower to a 1-state flower.

Just some notes for me to consider:

Just about every state could create its own cluster with as many flowers that exist in each one.

However, there are just some flower categories that exist in many states at the same time. Can they be classified in that segmented (statewide) cluster if their direct neighbors are vastly spread out?

One example to look at includes Hawaii. It's its own cluster but seems to have a similar number of flowers in the general grouping. Are they the same flowers? Are half of the flowers in Hawaii also in other states?

Another idea is that Jaccard is a harsh measurement tool. Does 1+0x69 differ enough from 2_0x68? Would something like cosine be able to take the vectors of each flower and group them closer?

Obviously thinking about a map oerlay might not work well. Its going to look like a giant blob over the entire continent and then four little blobs in Hawaii, Puerto Rico, Texas, and California.

What it's seeming to do is position its clusterings only at the extreme ends. If it doesn't exist in its own state or it doesnt exist in all states, its not useful. And that in itself isnt very useful.

I went on a walk to think a little more about the scenario.

The objective is to find the tightest 5 clusters with at least 1000 flowers in each. So while a flower might be found in more than one state, it wouldn't fit in a tight cluster. for every extra state the flower is found in, the cluster expands. Even just one flower in 10 states would seriously disproportionize the segmented space.

This is why we see the mega cluser containing all the flowers that exist in most states. It represents a cluster of its own, tightly fit to the flowers that exist in many regions and exluding more and more as the flowers are more sparse.

The representation of the clusters on a map isn't exactly accurate, but one that provides a good piece of insight. The data as I might see it in my mind is plotted on a map of north america, but the data as represented by the cluster is plotted in a 70-D space. So instead of the data looking like pieces of a puzzle, each state-sum for each flower represents something like a set of its own: "here are all the flowers located in 70 states, and here are all the flowers located in 69 states, and here are all the flowers located in 68 states..." These sets are grouped closer together as they sit on nearby planes, but as the flowers become more and more sporatic, it leaves that zone of influence. We start to see them floating around on the edges of the 70-D space not really relating to anything until it begins to converge again. When the sets start to reach the range of "and here are all the flowers located in this 3-state range, and here are all the flowers located in this 2-state range, and here are all the flowers in this one-state range" We start to see the flowers able to relate again.

I think the easiest to explain is the noise portion. It includes any points that would disproportionize the mega cluster or the micro clusters. I believe that we would see several more regional clusters, such as florida-georgia-alabama, if the cluster sizes could be smaller. For example, 'sabal palmetto' is a regional flower located in North Carolina and 6 surrounding states. It could represent a smaller cluster, but added to a larger one, it only widens the cluster space. That's what every other pseudo cluster would be doing. Unless there were 1000 flowers found in a 3-state region (excluding all other states), they only act to disrupt a cluster structure.

And this is probably why we see clusters a the extremes. It's a lot easier to relate a 70-state flower to a 69-state flower. It's much harder to relate a 3-state flower to a 2-state flower to a 1-state flower.

Although it has me thinking about the relationship between state counts. If a cluster centroid represents a 70-state flower, does a 69-state flower work because it matches? Obviously it would have to match if it included all the same states. but what about something a little less probable?

It would have to be iterative---something like "does the 69-s fit the 70-s? Does the 68-s fit the 69-s? does 67-s fit the 68-s?" This would keep going down to completely small points like "does the 7-s fit the 8-s?" As we go through these iterations, as soon as a flower-state matrix doesn't match the parent, it drops from the cluster. Eventually, only one 1-s would exist in the mega cluster. And all the others would be outside.

This reminds me a lot of the Apriori Algorithm. Side note

But really it helps me understand why some flowers would be included in the mega cluster when they feel regional.A combination of needing to tightly fit the cluster and needing to have 1000 flowers per cluster.

Thus, it has me thinking about a few things:
- what would happen if I changed the metric? (easy)
- what would happen if I changed the minimum cluster size? (easy)
- does apriori algorithmic preclustering do anything different? (difficult)

In [ ]:
from hdbscan.hdbscan_ import HDBSCAN
import matplotlib.pyplot as plt

clusterer= HDBSCAN(min_cluster_size= 1000,
                   min_samples=3,
                   metric= 'euclidean',
                   core_dist_n_jobs= -3
                   )

clusterer.fit(df)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


HDBSCAN(core_dist_n_jobs=-3, min_cluster_size=1000, min_samples=3)

In [ ]:
labels= clusterer.labels_

unique_labels, count= np.unique(labels, return_counts= True)
for label, cnt in zip(unique_labels, count):
  name= (f"Cluster {label}") if label != -1 else "Noise"
  print(f"{name}: {cnt} points")

Noise: 11671 points
Cluster 0: 5111 points
Cluster 1: 2014 points
Cluster 2: 2694 points
Cluster 3: 2053 points
Cluster 4: 3736 points
Cluster 5: 1808 points
Cluster 6: 1160 points
Cluster 7: 1866 points
Cluster 8: 2668 points


In [ ]:
cluster_id = -1

mask     = (labels == cluster_id)
df_clust = df.loc[mask]

state_counts = df_clust.sum(axis=0)   # Series indexed by state code

state_counts_sorted = state_counts.sort_values(ascending=False)

print(f"State counts for Cluster {cluster_id}:")
print(state_counts_sorted[:-2])
flower_counts = df.loc[mask].sum(axis=1)  # Series indexed by flower name

#print(flower_counts.nlargest(len(flower_counts)).mean())

State counts for Cluster -1:
ny        5485
pa        5319
il        5065
va        5026
nc        4922
          ... 
yt        1553
hi        1546
lb        1308
fraspm    1198
nu         769
Length: 68, dtype: int64


Okay so here's where I'm at. There's some interesting differences to see when applying euclidean compared to jaccard, where the former is actually grouping the clusters in ways that would make sense to a human brain. Obviously the rubric says no euclidean, so I'm also interested in cosine. However, I'm not sure why I'm finding resistance to implementing it into HDBSCAN (the unique library). Im considering two options:
1. Upload the sklearn HDBSCAN and see if that will take cosine
2. Stick to Jaccard as my main test and mention the difference when using euclidean

Honestly, the first feels easiest but We'll see what I'm feeling.

When I think more about euclidean, I when through some phases. At the beginningI looked at it and the clusters resembled actual regions in the united states. Cool. but then I remembered to take a step back from the 2D space of a map and look at the clusters like I would with Jaccard. Sure, it might include several more flowers in those tighter regions, but it's also going to include positions well outside of those regions. What does that do to the cluster field? I think in general we start to see clusters with more outliers and less tight borders. Is that what we want?

Then, I thought about how euclidean might be weighing these points. for every 2000 counts in Utah, there's one point in Louisiana. Is that truly affecting the cluster border in a significant manner?

One thing I think about is what information do we ascertain from the clusters. In Jaccard, do we actually get a tight cluster of flowers in an area, or do we just get the grouping of flowers that are found in several areas? Do we really know from the cluster what flowers are where if the cluster just includes all the flowers located everywhere? Same goes with the micro clusters. Do they tell us anything about the data if they only include data from one state?

I considered the split of flowers in clusters and in noise. This is more arbitrary than it feels because they're still technically in a cluster, just not in the clusters we've defined. If we altered the cluster miniimum to include less points, we might see a lot more clusters and much less noise. So they still exist in a hierarchy although in this plane they aren't being represented.

So I just tested cosine, and it looks like it crashes the session without mini batching. So here's what I can do:
- consider weighting the points for jaccard so rare but telling points better influence cluster borders
- leave the experiment as is and explain what weighting might do to jaccard
- Additionally, explore how to determine min_samples

In [ ]:
from hdbscan.hdbscan_ import HDBSCAN
import matplotlib.pyplot as plt

clusterer= HDBSCAN(min_cluster_size= 1000,
                   min_samples=3,
                   metric= 'rogerstanimoto',
                   core_dist_n_jobs= -1
                   )

clusterer.fit(df)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


HDBSCAN(core_dist_n_jobs=-1, metric='rogerstanimoto', min_cluster_size=1000,
        min_samples=3)

In [ ]:
labels= clusterer.labels_

unique_labels, count= np.unique(labels, return_counts= True)
for label, cnt in zip(unique_labels, count):
  name= (f"Cluster {label}") if label != -1 else "Noise"
  print(f"{name}: {cnt} points")

Noise: 11671 points
Cluster 0: 5111 points
Cluster 1: 2014 points
Cluster 2: 2694 points
Cluster 3: 2053 points
Cluster 4: 3736 points
Cluster 5: 1808 points
Cluster 6: 1160 points
Cluster 7: 1866 points
Cluster 8: 2668 points


Okay, I like Rogers-Tanimoto and I'll talk more about this. My next concern is mitigating the single-linkage that comes native with HDBSCAN. To do this' I'm going to modify the individual connectivity between each point. I'll do this with parameters including min_samples, leaves, and small epsilons. Let's do that.

In [ ]:
from hdbscan.hdbscan_ import HDBSCAN
import matplotlib.pyplot as plt

clusterer= HDBSCAN(min_cluster_size= 1000,
                   min_samples= 1050,
                   metric= 'rogerstanimoto',
                   core_dist_n_jobs= -1,
                   )

clusterer.fit(df)

labels= clusterer.labels_

unique_labels, count= np.unique(labels, return_counts= True)
for label, cnt in zip(unique_labels, count):
  name= (f"Cluster {label}") if label != -1 else "Noise"
  print(f"{name}: {cnt} points")

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Noise: 22326 points
Cluster 0: 2577 points
Cluster 1: 1670 points
Cluster 2: 1294 points
Cluster 3: 1599 points
Cluster 4: 3419 points
Cluster 5: 1896 points


In [ ]:
cluster_id = 3

mask     = (labels == cluster_id)
df_clust = df.loc[mask]

state_counts = df_clust.sum(axis=0)   # Series indexed by state code

state_counts_sorted = state_counts.sort_values(ascending=False)

#print(flower_counts.nlargest(len(flower_counts)).mean())
state_counts_sorted.head(10)

,0
pr,2840
vi,984
fl,570
la,7
ga,3
al,2
nj,1
ny,1
dc,1
hi,1


I think what I am learning about these clusters is that it generally doesn't like having flowers from multiple states. And that's because in a 70-D space, multiple states means further distances. Imagine CA -> CA/AZ -> CA/OR. And then take it even further and place it next to CA/OR/ID which might be totally out there. Right? Because then we have to say "well how close are all these OR/ID flowers and how about these OR/ID/MO flowers?"
The whole thing just gets messier as you collect multiple states. And that's not to say we cant have different clusters of different sizes representing different things. It's just that under the parameters of the project, single states make the tightest clusters.

In [ ]:
cluster_series= pd.Series(labels, index= df.index, name= "cluster")

flower_name= "sabal palmetto"
cluster_id= cluster_series.loc[flower_name]

print(f"Flower '{flower_name}' is in Cluster {cluster_id}")
sp= df.loc[flower_name].sort_values(ascending=False)
print(sp.head(10))


Flower 'sabal palmetto' is in Cluster 3
al    1
nc    1
la    1
ms    1
ga    1
fl    1
sc    1
bc    0
ar    0
ak    0
Name: sabal palmetto, dtype: int64
